In [2]:
#datetime.dt.hour()

### YELLOW LINE

In [3]:
import geopandas as gpd 
yellow_stop =  gpd.read_parquet('station-id-map.parquet')

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [4]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'red'
yellow_ids = gdf[gdf['line'] == 'Y']

# Sort the 'red' rows by the 'size' column
yellow_ids_sorted = yellow_ids.sort_values(by='CurrentStationId')

# Print the sorted 'red' rows

yellow_ids_sorted.head(10)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
599,30175,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,1,POINT (-87.67289 42.01906),4232,Y,91020,42.01906,-87.67289,1
600,30297,NaN,Oakton-Skokie,None,42.026243,-87.747221,0,41680,1,Y,Y,1,POINT (-87.74722 42.02624),27,Y,91058,42.02686,-87.74745,1
598,30026,NaN,Dempster-Skokie,None,42.038951,-87.751919,0,40140,1,Y,Y,1,POINT (-87.75192 42.03895),699,Y,91078,42.03895,-87.75192,1
651,30298,NaN,Oakton-Skokie,None,42.026243,-87.747221,0,41680,1,Y,Y,5,POINT (-87.74722 42.02624),10401,Y,95021,0.00000,0.00000,5
650,30176,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,5,POINT (-87.67289 42.01906),10401,Y,95021,0.00000,0.00000,5
649,30027,NaN,Dempster-Skokie,None,42.038951,-87.751919,0,40140,1,Y,Y,5,POINT (-87.75192 42.03895),10401,Y,95021,0.00000,0.00000,5
658,30027,NaN,Dempster-Skokie,None,42.038951,-87.751919,0,40140,1,Y,Y,5,POINT (-87.75192 42.03895),93223,Y,95030,0.00000,0.00000,5
659,30176,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,5,POINT (-87.67289 42.01906),93223,Y,95030,0.00000,0.00000,5
660,30298,NaN,Oakton-Skokie,None,42.026243,-87.747221,0,41680,1,Y,Y,5,POINT (-87.74722 42.02624),93223,Y,95030,0.00000,0.00000,5
625,30027,NaN,Dempster-Skokie,None,42.038951,-87.751919,0,40140,1,Y,Y,5,POINT (-87.75192 42.03895),2353,Y,95035,0.00000,0.00000,5


### How long between Howard and Oakton-Skokie?

In [5]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet')

In [6]:
station_id_map[(station_id_map.line == 'Y') & station_id_map.stop_name.isin(['Howard', 'Oakton-Skokie'])].head(5)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
599,30175,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,1,POINT (-87.67289 42.01906),4232,Y,91020,42.01906,-87.67289,1
600,30297,NaN,Oakton-Skokie,None,42.026243,-87.747221,0,41680,1,Y,Y,1,POINT (-87.74722 42.02624),27,Y,91058,42.02686,-87.74745,1
602,30176,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,5,POINT (-87.67289 42.01906),1079,Y,95040,0.00000,0.00000,5
603,30298,NaN,Oakton-Skokie,None,42.026243,-87.747221,0,41680,1,Y,Y,5,POINT (-87.74722 42.02624),1079,Y,95040,0.00000,0.00000,5
605,30176,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,5,POINT (-87.67289 42.01906),1308,Y,95045,0.00000,0.00000,5


In [7]:
# In previous result, scroll over to see CurrentStationId -> 91020, 91058
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')

In [8]:
edf.query('RunNumber == 593 and Datetime > "2022-03-01 00:53:00"')
# Now, look at a RunNumber on the Yellow Line and just those stations in the "1" direction
edf.query('RunNumber == 593 and CurrentStationId.isin([91020, 91058])')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
27,Y,2022-03-01 00:00:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
101,Y,2022-03-01 00:00:21,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
6688,Y,2022-03-01 00:30:42,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
86483,Y,2022-03-01 12:27:42,42.01906,-87.67289,593,91020,95090,2.464156,Yel,to,Skokie,False,ttpm_predslip_yellow,
87487,Y,2022-03-01 12:33:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420480,Y,2022-03-07 22:33:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
907,Y,2022-03-07 23:01:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
7863,Y,2022-03-07 23:29:01,42.01906,-87.67289,593,91020,95090,2.464156,Yel,to,Skokie,False,ttpm_predslip_yellow,
9063,Y,2022-03-07 23:34:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,


In [9]:
# We have multiple entries for some stations (the train might stay there for a minute)
yellow_howard_yellow_oakton_skokie = edf.query('RunNumber == 593 and CurrentStationId.isin([91020, 91058])')
yellow_howard_yellow_oakton_skokie_dedup = yellow_howard_yellow_oakton_skokie[yellow_howard_yellow_oakton_skokie.CurrentStationId.diff() != 0]

In [10]:
yellow_howard_yellow_oakton_skokie_dedup.Datetime.diff()

27                   NaT
86483    0 days 12:27:41
87487    0 days 00:05:19
92868    0 days 00:27:00
93885    0 days 00:04:41
               ...      
408338   0 days 00:05:00
419286   0 days 00:52:41
420405   0 days 00:05:00
7863     0 days 00:56:19
9063     0 days 00:05:00
Name: Datetime, Length: 105, dtype: timedelta64[ns]

In [11]:
# the actual times are betweeen 31640 and 31600 so we want every other one, starting with 1
yellow_howard_yellow_oakton_skokie_dedup.Datetime.diff().iloc[1::2] # 1 is the start, empty is the end and 2 is the step size

86483    0 days 12:27:41
92868    0 days 00:27:00
98688    0 days 00:21:39
106159   0 days 00:25:00
114249   0 days 00:24:40
191368   0 days 06:02:20
203210   0 days 00:23:41
209944   0 days 00:24:00
233849   0 days 01:27:00
336448   0 days 12:49:59
365954   0 days 02:01:20
442952   0 days 00:24:39
461944   0 days 01:26:40
468914   0 days 00:23:00
486401   0 days 01:00:00
494221   0 days 00:23:40
502081   0 days 00:26:40
16818    0 days 11:48:21
22757    0 days 00:24:00
30396    0 days 00:26:41
37648    0 days 00:21:40
121080   0 days 05:55:00
126872   0 days 00:23:40
132872   0 days 00:24:20
145985   0 days 00:54:40
177716   0 days 01:54:40
275386   0 days 12:55:20
282269   0 days 00:25:00
289384   0 days 00:22:59
297836   0 days 00:26:40
380069   0 days 06:24:19
386137   0 days 00:24:00
392553   0 days 00:24:20
399625   0 days 00:24:40
407181   0 days 00:24:20
423342   0 days 00:55:39
430867   0 days 00:26:19
48856    0 days 19:59:40
70281    0 days 02:22:00
100565   0 days 04:11:00


### How long from Oakton-Skokie to Dempster-Skokie?

In [12]:
# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Yellow (Oakton-Skokie to Dempster-Skokie)
station_id_map[(station_id_map.line == 'Y') & station_id_map.stop_name.isin(['Oakton-Skokie, Dempster-Skokie'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode


In [13]:
# In previous result, scroll over to see CurrentStationId -> 91058, 91078
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')

edf.query('RunNumber == 593 and Datetime > "2022-03-01 00:53:00"')

# Now, look at a RunNumber on the Brown Line and just those stations in the "1" direction
edf.query('RunNumber == 593 and CurrentStationId.isin([91058, 91078])')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
27,Y,2022-03-01 00:00:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
101,Y,2022-03-01 00:00:21,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
699,Y,2022-03-01 00:03:01,42.03895,-87.75192,593,91078,95090,5.069098,Yel,to,Skokie,False,ttpm_predslip_yellow,
776,Y,2022-03-01 00:03:21,42.03895,-87.75192,593,91078,95090,5.069098,Yel,to,Skokie,False,ttpm_predslip_yellow,
6688,Y,2022-03-01 00:30:42,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,Y,2022-03-07 23:04:01,42.03895,-87.75192,593,91078,95090,5.069098,Yel,to,Skokie,False,ttpm_predslip_yellow,
9063,Y,2022-03-07 23:34:01,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
9144,Y,2022-03-07 23:34:21,42.02686,-87.74745,593,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
9701,Y,2022-03-07 23:36:42,42.03895,-87.75192,593,91078,95090,5.069098,Yel,to,Skokie,False,ttpm_predslip_yellow,


In [14]:
# We have multiple entries for some stations (the train might stay there for a minute)
yellow_oakton_skokie_yellow_dempster_skokie = edf.query('RunNumber == 593 and CurrentStationId.isin([91058, 91078])')
yellow_oakton_skokie_yellow_dempster_skokie_dedup = yellow_oakton_skokie_yellow_dempster_skokie[yellow_oakton_skokie_yellow_dempster_skokie.CurrentStationId.diff() != 0]

In [15]:
# compute the difference
yellow_oakton_skokie_yellow_dempster_skokie_dedup.Datetime.diff()

27                   NaT
699      0 days 00:03:00
6688     0 days 00:27:41
87929    0 days 12:04:40
93885    0 days 00:29:20
               ...      
421018   0 days 00:02:40
907      0 days 00:25:39
1561     0 days 00:02:40
9063     0 days 00:30:20
9701     0 days 00:02:41
Name: Datetime, Length: 172, dtype: timedelta64[ns]

In [16]:
# the actual times are betweeen 91058 and 91078 so we want every other one, starting with 1
yellow_oakton_skokie_yellow_dempster_skokie_dedup.Datetime.diff().iloc[1::2] # 1 is the start, empty is the end and 2 is the step size

699      0 days 00:03:00
87929    0 days 12:04:40
94543    0 days 00:03:00
100449   0 days 00:02:21
108092   0 days 00:02:00
               ...      
408853   0 days 07:04:00
413983   0 days 00:02:40
421018   0 days 00:02:40
1561     0 days 00:02:40
9701     0 days 00:02:41
Name: Datetime, Length: 86, dtype: timedelta64[ns]

### Yellow Predictions

In [17]:
import pandas as pd
pdf = pd.read_parquet('2022-03-preds-start.parquet')

In [18]:
# want a map from parent_station (aka ParentStop in the pdf) to CurrentStationId for Yellow Line
# station_id_map.CurrentStationId // 1000 == 91 find the stations on the brown line (3) going in direction (1) (Howard)
yellow_entries = station_id_map[(station_id_map.line == 'Y') & (station_id_map.CurrentStationId // 1000 == 91)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [19]:
yellow_entries.query('stop_name == "Howard"')

,stop_name,parent_station,CurrentStationId
599,Howard,40900,91020


In [20]:
# Get run 593 again
yellow = pdf.query("RunNumber == 593")

In [21]:
ymap = yellow_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()

In [22]:
# map the ParentStop through the bmap to get the StationId for the Prediction
yellow_with_id = yellow.assign(PredictionStationId=yellow.ParentStop.map(ymap))

In [23]:
# look at the normal prediction for 91020 to 91058
yellow_with_id[(yellow_with_id.CurrentStationId == 91020) & (yellow_with_id.PredictionStationId == 91058)]

,Line,Datetime,CurrentStationId,RunNumber,StopOrder,ParentStop,StopName,Prediction,PredictionStationId
410089,Y,2022-03-01 12:27:42,91020,593,0,41680,Oakton-Skokie,5,91058
443201,Y,2022-03-01 13:00:01,91020,593,0,41680,Oakton-Skokie,4,91058
880735,Y,2022-03-01 20:34:01,91020,593,0,41680,Oakton-Skokie,5,91058
1528941,Y,2022-03-02 12:26:21,91020,593,0,41680,Oakton-Skokie,6,91058
517225,Y,2022-03-03 19:57:01,91020,593,0,41680,Oakton-Skokie,5,91058
1640618,Y,2022-03-04 20:58:01,91020,593,0,41680,Oakton-Skokie,5,91058
1667173,Y,2022-03-04 21:27:01,91020,593,0,41680,Oakton-Skokie,5,91058
206431,Y,2022-03-05 20:34:01,91020,593,0,41680,Oakton-Skokie,6,91058
872729,Y,2022-03-06 22:22:41,91020,593,0,41680,Oakton-Skokie,4,91058
878609,Y,2022-03-06 22:33:42,91020,593,0,41680,Oakton-Skokie,5,91058


### Oakton-Skokie to Dempster-Skokie Prediction

In [24]:
# want a map from parent_station (aka ParentStop in the pdf) to CurrentStationId for Yellow Line
# station_id_map.CurrentStationId // 1000 == 91 find the stations on the brown line (3) going in direction (1) (Howard)
yellow_entries = station_id_map[(station_id_map.line == 'Y') & (station_id_map.CurrentStationId // 1000 == 91)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [25]:
yellow_entries.query('stop_name == "Oakton-Skokie"')

,stop_name,parent_station,CurrentStationId
600,Oakton-Skokie,41680,91058


In [26]:
# Get run 593 again
yellow = pdf.query("RunNumber == 593")

In [27]:
ymap = yellow_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()

In [28]:
# map the ParentStop through the bmap to get the StationId for the Prediction
yellow_with_id = yellow.assign(PredictionStationId=yellow.ParentStop.map(ymap))

In [29]:
# look at the normal prediction for 91058 to 91078
yellow_with_id[(yellow_with_id.CurrentStationId == 91058) & (yellow_with_id.PredictionStationId == 91078)]

,Line,Datetime,CurrentStationId,RunNumber,StopOrder,ParentStop,StopName,Prediction,PredictionStationId
202,Y,2022-03-01 00:00:01,91058,593,1,40140,Dempster-Skokie,3,91078
798924,Y,2022-03-04 00:32:21,91058,593,1,40140,Dempster-Skokie,4,91078
1166711,Y,2022-03-04 12:34:22,91058,593,1,40140,Dempster-Skokie,3,91078
1619677,Y,2022-03-04 20:34:21,91058,593,1,40140,Dempster-Skokie,4,91078
1645233,Y,2022-03-04 21:03:01,91058,593,1,40140,Dempster-Skokie,3,91078


### Attempts at sorting differently

In [30]:
import geopandas as gpd

df1 = yellow_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = yellow_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'Dempster-Skokie': [91078, 95021, 95030, 95035, 95040, 95041, 95042, 95045, 95048, 95052, 95058, 95060, 95062, 95064, 95066, 95068, 95070, 95072, 95075, 95078, 95080], 'Howard': [91020, 95021, 95030, 95035, 95040, 95041, 95042, 95045, 95048, 95052, 95058, 95060, 95062, 95064, 95066, 95068, 95070, 95072, 95075, 95078, 95080], 'Oakton-Skokie': [91058, 95021, 95030, 95035, 95040, 95041, 95042, 95045, 95048, 95052, 95058, 95060, 95062, 95064, 95066, 95068, 95070, 95072, 95075, 95078, 95080]}


In [31]:
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')

In [76]:
station_id_map[(station_id_map.line == 'Y') & station_id_map.stop_name.isin(['Howard','Oakton-Skokie'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode


In [33]:
import itertools



### BLUE LINE

In [34]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'red'
blue_ids = gdf[gdf['line'] == 'B']

# Sort the 'red' rows by the 'size' column
blue_ids_sorted = blue_ids.sort_values(by='CurrentStationId')

# Print the sorted 'red' rows

blue_ids_sorted.head(60)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
17,30076,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,1,POINT (-87.81732 41.87426),31,B,11020,41.87426,-87.81732,1
52,30189,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,1,POINT (-87.80696 41.87349),476,B,11040,41.87349,-87.80696,1
3,30034,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,1,POINT (-87.79160 41.87211),329,B,11060,41.87211,-87.79160,1
0,30001,NaN,Austin-Blue,None,41.870851,-87.776812,0,40010,0,Blue,B,1,POINT (-87.77681 41.87085),3941,B,11080,41.87089,-87.77681,1
51,30187,NaN,Cicero-Forest Park,None,41.871574,-87.745154,0,40970,0,Blue,B,1,POINT (-87.74515 41.87157),5024,B,11100,41.87161,-87.74515,1
50,30179,NaN,Pulaski-Forest Park,None,41.873797,-87.725663,0,40920,0,Blue,B,1,POINT (-87.72566 41.87380),3647,B,11120,41.87389,-87.72566,1
9,30048,NaN,Kedzie-Homan,None,41.874341,-87.706040,0,40250,1,Blue,B,1,POINT (-87.70604 41.87434),6552,B,11140,41.87442,-87.70605,1
4,30042,NaN,Western-Forest Park,None,41.875478,-87.688436,0,40220,0,Blue,B,1,POINT (-87.68844 41.87548),7177,B,11160,41.87553,-87.68844,1
34,30157,NaN,Illinois Medical District,None,41.875706,-87.673932,0,40810,1,Blue,B,1,POINT (-87.67393 41.87571),37,B,11180,41.87564,-87.67393,1
23,30092,NaN,Racine,None,41.875920,-87.659458,0,40470,0,Blue,B,1,POINT (-87.65946 41.87592),412564,B,11201,41.87589,-87.65946,1


In [77]:
blue_ids_sorted.drop_duplicates("stop_name")

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
17,30076,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,1,POINT (-87.81732 41.87426),31,B,11020,41.87426,-87.81732,1
52,30189,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,1,POINT (-87.80696 41.87349),476,B,11040,41.87349,-87.80696,1
3,30034,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,1,POINT (-87.79160 41.87211),329,B,11060,41.87211,-87.79160,1
0,30001,NaN,Austin-Blue,None,41.870851,-87.776812,0,40010,0,Blue,B,1,POINT (-87.77681 41.87085),3941,B,11080,41.87089,-87.77681,1
51,30187,NaN,Cicero-Forest Park,None,41.871574,-87.745154,0,40970,0,Blue,B,1,POINT (-87.74515 41.87157),5024,B,11100,41.87161,-87.74515,1
50,30179,NaN,Pulaski-Forest Park,None,41.873797,-87.725663,0,40920,0,Blue,B,1,POINT (-87.72566 41.87380),3647,B,11120,41.87389,-87.72566,1
9,30048,NaN,Kedzie-Homan,None,41.874341,-87.706040,0,40250,1,Blue,B,1,POINT (-87.70604 41.87434),6552,B,11140,41.87442,-87.70605,1
4,30042,NaN,Western-Forest Park,None,41.875478,-87.688436,0,40220,0,Blue,B,1,POINT (-87.68844 41.87548),7177,B,11160,41.87553,-87.68844,1
34,30157,NaN,Illinois Medical District,None,41.875706,-87.673932,0,40810,1,Blue,B,1,POINT (-87.67393 41.87571),37,B,11180,41.87564,-87.67393,1
23,30092,NaN,Racine,None,41.875920,-87.659458,0,40470,0,Blue,B,1,POINT (-87.65946 41.87592),412564,B,11201,41.87589,-87.65946,1


### Datetime Calc Function

In [35]:
import pandas as pd

two_stations = edf.query('RunNumber == 120 and CurrentStationId.isin([11040, 11060])')
   
two_stations_dedup = two_stations[two_stations.CurrentStationId.diff() != 0].assign(PrevStationId=lambda edf: edf.CurrentStationId.shift())

two_stations_dedup = two_stations[two_stations.CurrentStationId.diff() != 0].assign(
    PrevStationId=lambda edf: edf.CurrentStationId.shift(),
    Time=lambda edf: edf.Datetime.diff()
)

In [36]:
two_stations_dedup.query('CurrentStationId == 11040 & PrevStationId == 11060').Time

370962   3 days 04:00:40
50823    1 days 00:34:40
80787    0 days 03:32:41
Name: Time, dtype: timedelta64[ns]

In [37]:
two_stations_dedup.groupby(['PrevStationId','CurrentStationId']).Time.describe() # or .mean() or .describe()

,,count,mean,std,min,25%,50%,75%,max
PrevStationId,CurrentStationId,,,,,,,,
11040.0,11060,4,0 days 00:01:54.500000,0 days 00:00:18.982448033,0 days 00:01:39,0 days 00:01:39.750000,0 days 00:01:50,0 days 00:02:04.750000,0 days 00:02:19
11060.0,11040,3,1 days 10:42:40.333333333,1 days 13:16:50.931746759,0 days 03:32:41,0 days 14:03:40.500000,1 days 00:34:40,2 days 02:17:40,3 days 04:00:40


In [38]:
two_stations_dedup.groupby(['PrevStationId','CurrentStationId']).Time.median() # or .mean() or .describe()

PrevStationId  CurrentStationId
11040.0        11060              0 days 00:01:50
11060.0        11040              1 days 00:34:40
Name: Time, dtype: timedelta64[ns]

In [39]:
import geopandas as gpd

df2 = blue_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = blue_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'Addison-Blue': [11540, 15180], 'Austin-Blue': [11080, 15640], 'Belmont-Blue': [11520, 15200], "California-O'Hare": [11480, 15240], 'Chicago-Blue': [11400, 15320], 'Cicero-Forest Park': [11100, 15620], 'Clark/Lake (Subway)': [11360, 15360], 'Clinton-Blue': [11260, 15460], 'Cumberland': [11622, 11631, 11633, 11640, 15062, 15065, 15067, 15074, 15076, 15078, 15080], "Damen-O'Hare": [11440, 15280], 'Division': [11420, 15281, 15300], 'Forest Park': [11020, 15695], 'Grand-Blue': [11380, 15340], 'Harlem-Forest Park': [11040, 15680], "Harlem-O'Hare": [11604, 11605, 11620, 15082, 15083, 15085, 15087, 15093, 15095, 15097, 15100], 'Illinois Medical District': [11180, 15540], 'Irving Park-Blue': [11550, 11560, 15160], 'Jackson-Blue': [11300, 15420], 'Jefferson Park': [11591, 11593, 11598, 11599, 11600, 15102, 15105, 15113, 15114, 15115, 15116, 15120], 'Kedzie-Homan': [11140, 15578], 'LaSalle': [11280, 15440], 'Logan Square': [11500, 15220, 15231], 'Monroe-Blue': [11320, 15400], 'Montrose-Blue': [

In [40]:
{k: v for k, v in sorted(my_dict.items(), key=lambda item: item[1])}

{'Forest Park': [11020, 15695],
 'Harlem-Forest Park': [11040, 15680],
 'Oak Park-Blue': [11060, 15660],
 'Austin-Blue': [11080, 15640],
 'Cicero-Forest Park': [11100, 15620],
 'Pulaski-Forest Park': [11120, 15600],
 'Kedzie-Homan': [11140, 15578],
 'Western-Forest Park': [11160, 15560],
 'Illinois Medical District': [11180, 15540],
 'Racine': [11201, 11202, 11203, 11206, 11220, 15500],
 'UIC-Halsted': [11228, 11230, 11232, 11235, 11240, 15480],
 'Clinton-Blue': [11260, 15460],
 'LaSalle': [11280, 15440],
 'Jackson-Blue': [11300, 15420],
 'Monroe-Blue': [11320, 15400],
 'Washington-Blue': [11340, 15380],
 'Clark/Lake (Subway)': [11360, 15360],
 'Grand-Blue': [11380, 15340],
 'Chicago-Blue': [11400, 15320],
 'Division': [11420, 15281, 15300],
 "Damen-O'Hare": [11440, 15280],
 "Western-O'Hare": [11460, 15260],
 "California-O'Hare": [11480, 15240],
 'Logan Square': [11500, 15220, 15231],
 'Belmont-Blue': [11520, 15200],
 'Addison-Blue': [11540, 15180],
 'Irving Park-Blue': [11550, 11560, 

In [41]:
edf.query("RunNumber == 120").head(5)

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
29,B,2022-03-01 00:00:01,41.97208,-87.76551,120,11602,11700,2.743046,Blu,to,O'Hare,False,ttpm_predslip_blue,
103,B,2022-03-01 00:00:21,41.98232,-87.80890,120,11604,11700,3.151556,Blu,to,O'Hare,False,ttpm_predslip_blue,
176,B,2022-03-01 00:00:41,41.97496,-87.77076,120,11607,11700,2.623162,Blu,to,O'Hare,False,ttpm_predslip_blue,
250,B,2022-03-01 00:01:02,41.97580,-87.77222,120,11608,11700,2.622869,Blu,to,O'Hare,False,ttpm_predslip_blue,
324,B,2022-03-01 00:01:22,41.97800,-87.77609,120,11610,11700,2.623537,Blu,to,O'Hare,False,ttpm_predslip_blue,


In [42]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet')

# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Blue Line (Forest Park to Harlem-Forest Park)
station_id_map[(station_id_map.line == 'B') & station_id_map.stop_name.isin(['Forest Park', 'Harlem-Forest Park'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
17,30076,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,1,POINT (-87.81732 41.87426),31,B,11020,41.87426,-87.81732,1
52,30189,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,1,POINT (-87.80696 41.87349),476,B,11040,41.87349,-87.80696,1
81,30077,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,5,POINT (-87.81732 41.87426),1941,B,15695,41.87376,-87.80963,5
112,30190,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,5,POINT (-87.80696 41.87349),1634,B,15680,41.87349,-87.80696,5


In [43]:
# In previous result, scroll over to see CurrentStationId -> 11020, 11040
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 120 and Datetime > "2022-03-01 00:53:00"').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
139466,B,2022-03-01 16:07:21,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139544,B,2022-03-01 16:07:41,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139620,B,2022-03-01 16:08:01,41.87376,-87.80970,120,11035,11700,6.182494,Blu,to,O'Hare,False,ttpm_predslip_blue,
139694,B,2022-03-01 16:08:22,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139767,B,2022-03-01 16:08:42,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,


In [44]:
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 120')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
29,B,2022-03-01 00:00:01,41.97208,-87.76551,120,11602,11700,2.743046,Blu,to,O'Hare,False,ttpm_predslip_blue,
103,B,2022-03-01 00:00:21,41.98232,-87.80890,120,11604,11700,3.151556,Blu,to,O'Hare,False,ttpm_predslip_blue,
176,B,2022-03-01 00:00:41,41.97496,-87.77076,120,11607,11700,2.623162,Blu,to,O'Hare,False,ttpm_predslip_blue,
250,B,2022-03-01 00:01:02,41.97580,-87.77222,120,11608,11700,2.622869,Blu,to,O'Hare,False,ttpm_predslip_blue,
324,B,2022-03-01 00:01:22,41.97800,-87.77609,120,11610,11700,2.623537,Blu,to,O'Hare,False,ttpm_predslip_blue,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91155,B,2022-03-06 01:49:21,41.98384,-87.84917,120,11650,11660,3.178393,Blu,to,Rosemont,False,ttpm_predslip_blue,
91195,B,2022-03-06 01:49:41,41.98351,-87.85939,120,11651,11660,3.176772,Blu,to,Rosemont,False,ttpm_predslip_blue,
91235,B,2022-03-06 01:50:01,41.98366,-87.85574,120,11655,11660,3.181652,Blu,to,Rosemont,False,ttpm_predslip_blue,
91275,B,2022-03-06 01:50:21,41.98363,-87.85659,120,11658,11660,3.181652,Blu,to,Rosemont,False,ttpm_predslip_blue,


In [45]:
# Now, look at a RunNumber on the Brown Line and just those stations in the "1" direction
BlueForestPark =  11020 #BlueForestPark
BlueHarlemForestPark = 11040 #BlueHarlem-ForestPark
edf.query(f'RunNumber == 120 and CurrentStationId.isin([{BlueForestPark}, {BlueHarlemForestPark}])').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
139466,B,2022-03-01 16:07:21,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139544,B,2022-03-01 16:07:41,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139694,B,2022-03-01 16:08:22,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139767,B,2022-03-01 16:08:42,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139840,B,2022-03-01 16:09:01,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,


In [46]:
# We have multiple entries for some stations (the train might stay there for a minute)
ForestParkToHarlemForestPark = edf.query(f'RunNumber == 120 and CurrentStationId.isin([{BlueForestPark}, {BlueHarlemForestPark}])')
ForestParkToHarlemForestPark_dedup = ForestParkToHarlemForestPark[ForestParkToHarlemForestPark.CurrentStationId.diff() != 0]

# compute the difference
ForestParkToHarlemForestPark_dedup.Datetime.diff()

139466               NaT
139694   0 days 00:01:01
370768   3 days 04:02:00
370962   0 days 00:00:59
50632    1 days 00:35:20
50823    0 days 00:01:20
80659    0 days 03:33:20
80787    0 days 00:01:01
Name: Datetime, dtype: timedelta64[ns]

In [47]:
# the actual times are betweeen 31735 and 31720 so we want every other one, starting with 2
ForestParkToHarlemForestPark_dedup.Datetime.diff().iloc[1::2] # 2 is the start, empty is the end and 2 is the step size

139694   0 days 00:01:01
370962   0 days 00:00:59
50823    0 days 00:01:20
80787    0 days 00:01:01
Name: Datetime, dtype: timedelta64[ns]

In [48]:
# the actual times are betweeen 31735 and 31720 so we want every other one, starting with 2
ForestParkToHarlemForestPark_dedup.Datetime.diff().iloc[1::2] # 2 is the start, empty is the end and 2 is the step size

139694   0 days 00:01:01
370962   0 days 00:00:59
50823    0 days 00:01:20
80787    0 days 00:01:01
Name: Datetime, dtype: timedelta64[ns]

### Harlem to Oak Park

In [49]:
# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Brown Line (Irving Park-Brown to Damen-Brown)
station_id_map[(station_id_map.line == 'B') & station_id_map.stop_name.isin(['Harlem-Forest Park','Oak Park-Blue'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
3,30034,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,1,POINT (-87.79160 41.87211),329,B,11060,41.87211,-87.79160,1
52,30189,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,1,POINT (-87.80696 41.87349),476,B,11040,41.87349,-87.80696,1
67,30035,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,5,POINT (-87.79160 41.87211),41,B,15660,41.87211,-87.79160,5
112,30190,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,5,POINT (-87.80696 41.87349),1634,B,15680,41.87349,-87.80696,5


In [50]:
# Now, look at a RunNumber on the Blue Line and just those stations in the "1" direction
edf.query('RunNumber == 120 and CurrentStationId.isin([11040, 11060])')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
139694,B,2022-03-01 16:08:22,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139767,B,2022-03-01 16:08:42,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139840,B,2022-03-01 16:09:01,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
140215,B,2022-03-01 16:10:41,41.87211,-87.79160,120,11060,11700,6.195430,Blu,to,O'Hare,False,ttpm_predslip_blue,
370962,B,2022-03-04 20:11:21,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
371343,B,2022-03-04 20:13:21,41.87211,-87.79160,120,11060,11700,6.195430,Blu,to,O'Hare,False,ttpm_predslip_blue,
371405,B,2022-03-04 20:13:41,41.87211,-87.79160,120,11060,11700,6.195430,Blu,to,O'Hare,False,ttpm_predslip_blue,
371466,B,2022-03-04 20:14:01,41.87211,-87.79160,120,11060,11700,6.195430,Blu,to,O'Hare,False,ttpm_predslip_blue,
50823,B,2022-03-05 20:48:01,41.87349,-87.80696,120,11040,11660,6.183922,Blu,to,Rosemont,False,ttpm_predslip_blue,
50871,B,2022-03-05 20:48:21,41.87349,-87.80696,120,11040,11660,6.183922,Blu,to,Rosemont,False,ttpm_predslip_blue,


In [51]:
# We have multiple entries for some stations (the train might stay there for a minute)
harlem_oakpark = edf.query('RunNumber == 120 and CurrentStationId.isin([11040, 11060])')
harlem_oakpark_dedup = harlem_oakpark[harlem_oakpark.CurrentStationId.diff() != 0]

### Prediction

In [52]:
# want a map from parent_station (aka ParentStop in the pdf) to CurrentStationId for Brown Line
# station_id_map.CurrentStationId // 1000 == 11 find the stations on the blue line (3) going in direction (1) (Harlem)
blue_entries = station_id_map[(station_id_map.line == 'B') & (station_id_map.CurrentStationId // 1000 == 11)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [53]:
bluemap = blue_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()

### RED LINE

In [54]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'red'
red_ids = gdf[gdf['line'] == 'R']

# Sort the 'red' rows by the 'size' column
red_ids_sorted = red_ids.sort_values(by='CurrentStationId')

# Print the sorted 'red' rows

red_ids_sorted.head()


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
382,30233,NaN,Fullerton,None,41.925051,-87.652866,0,41220,1,Red,R,1,POINT (-87.65287 41.92505),187778,R,31403,41.92505,-87.65287,1
384,30233,NaN,Fullerton,None,41.925051,-87.652866,0,41220,1,Red,R,1,POINT (-87.65287 41.92505),167642,R,31420,41.92505,-87.65287,1
394,30255,NaN,Belmont,None,41.939751,-87.653380,0,41320,1,Red,R,1,POINT (-87.65338 41.93975),177756,R,31468,41.93975,-87.65338,1
373,30173,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Red,R,1,POINT (-87.67289 42.01906),229567,R,71176,42.01906,-87.67289,1
371,30173,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Red,R,1,POINT (-87.67289 42.01906),231399,R,71300,42.01906,-87.67289,1


In [55]:
import geopandas as gpd

red_data = red_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = red_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'47th-Red': [81160, 85660], '63rd': [81120, 85700], '69th': [81100, 85720], '79th': [81080, 85740], '87th': [81060, 85760], '95th': [81040, 85775], 'Addison-Red': [81540, 85280], 'Argyle': [81620, 85170, 85200], 'Belmont': [31468, 81493, 81500, 85315], 'Bryn Mawr': [81642, 81650, 81659, 81660, 85142, 85150, 85156, 85159, 85160], 'Cermak-Chinatown': [81200, 81236, 85621], 'Chicago-Red': [81380, 85440], 'Clark/Division': [81400, 85420], 'Fullerton': [31403, 31420, 81465, 81480, 85322, 85340], 'Garfield-Red': [81140, 85680], 'Grand-Red': [81360, 85460], 'Granville': [81667, 81674, 81676, 81679, 81685, 81695, 81699, 81700, 85102, 85105, 85110, 85115, 85119, 85120], 'Harrison': [81260, 85560], 'Howard': [71176, 71300, 81675, 85060], 'Jackson-Red': [81280, 85540], 'Jarvis': [75220, 81760], 'Lake': [81340, 81354, 85480], 'Loyola': [81703, 81706, 81710, 81713, 81718, 81719, 81720, 85090, 85098, 85099, 85100], 'Monroe-Red': [81300, 85520], 'Morse': [81722, 81725, 81740, 85080], 'North/Clybourn

### BROWN LINE

In [56]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'brown'
brown_ids = gdf[gdf['line'] == 'T']

# Sort the 'brown' rows by the 'size' column
brown_ids_sorted = brown_ids.sort_values(by='CurrentStationId')

# Print the sorted 'brown' rows

brown_ids_sorted.head()


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
485,30142,NaN,Washington/Wells,None,41.882695,-87.633780,0,40730,1,Brn,T,1,POINT (-87.63378 41.88269),8466,T,31060,41.88269,-87.63385,1
462,30008,NaN,Quincy,None,41.878723,-87.633740,0,40040,1,Brn,T,1,POINT (-87.63374 41.87872),1660,T,31080,41.87872,-87.63374,1
464,30030,NaN,LaSalle/Van Buren,None,41.876800,-87.631739,0,40160,0,Brn,T,1,POINT (-87.63174 41.87680),1965,T,31100,41.87686,-87.63174,1
494,30165,NaN,Harold Washington Library,None,41.876862,-87.628196,0,40850,1,Brn,T,1,POINT (-87.62820 41.87686),2107,T,31120,41.87691,-87.62820,1
480,30131,NaN,Adams/Wabash,None,41.879507,-87.626037,0,40680,0,Brn,T,1,POINT (-87.62604 41.87951),286,T,31180,41.87950,-87.62610,1


In [57]:
import geopandas as gpd

brown_data = brown_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = brown_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'Adams/Wabash': [31180], 'Addison-Brown': [31580, 35180], 'Armitage': [31382, 31383, 31387, 31390, 31393, 31396, 31398, 31399, 31400, 35360], 'Belmont': [31475, 35280, 81500], 'Chicago-Brown': [31345, 31350, 31358, 31360, 35382, 35384, 35386, 35389, 35390, 35394, 35396, 35398, 35399, 35400], 'Clark/Lake (Elevated)': [31260], 'Damen-Brown': [31640, 35120], 'Diversey': [31438, 35320], 'Francisco': [31700, 35060], 'Fullerton': [31420, 35340], 'Harold Washington Library': [31120], 'Irving Park-Brown': [31600, 35160], 'Kedzie-Brown': [31720, 35040], 'Kimball': [31735, 35020], 'LaSalle/Van Buren': [31100], 'Merchandise Mart': [31291, 31340, 35405, 35414, 35418, 35420], 'Montrose-Brown': [31620, 35140], 'Paulina': [31560, 35200], 'Quincy': [31080], 'Rockwell': [31680, 35080], 'Sedgwick': [31363, 31367, 31369, 31371, 31373, 31377, 31379, 31380, 35363, 35367, 35369, 35374, 35376, 35378, 35379, 35380], 'Southport': [31540, 35220], 'State/Lake': [31240], 'Washington/Wabash': [31215], 'Washington

In [58]:
{k: v for k, v in sorted(my_dict.items(), key=lambda item: item[1])}

{'Washington/Wells': [31060],
 'Quincy': [31080],
 'LaSalle/Van Buren': [31100],
 'Harold Washington Library': [31120],
 'Adams/Wabash': [31180],
 'Washington/Wabash': [31215],
 'State/Lake': [31240],
 'Clark/Lake (Elevated)': [31260],
 'Merchandise Mart': [31291, 31340, 35405, 35414, 35418, 35420],
 'Chicago-Brown': [31345,
  31350,
  31358,
  31360,
  35382,
  35384,
  35386,
  35389,
  35390,
  35394,
  35396,
  35398,
  35399,
  35400],
 'Sedgwick': [31363,
  31367,
  31369,
  31371,
  31373,
  31377,
  31379,
  31380,
  35363,
  35367,
  35369,
  35374,
  35376,
  35378,
  35379,
  35380],
 'Armitage': [31382,
  31383,
  31387,
  31390,
  31393,
  31396,
  31398,
  31399,
  31400,
  35360],
 'Fullerton': [31420, 35340],
 'Diversey': [31438, 35320],
 'Wellington': [31455, 35300],
 'Belmont': [31475, 35280, 81500],
 'Southport': [31540, 35220],
 'Paulina': [31560, 35200],
 'Addison-Brown': [31580, 35180],
 'Irving Park-Brown': [31600, 35160],
 'Montrose-Brown': [31620, 35140],
 'Dam

### GREEN LINE

In [59]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'green'
green_ids = gdf[gdf['line'] == 'G']

# Sort the 'green' rows by the 'size' column
green_ids_sorted = green_ids.sort_values(by='CurrentStationId')

# Print the sorted 'green' rows
green_ids_sorted.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
145,30131,NaN,Adams/Wabash,None,41.879507,-87.626037,0,40680,0,G,G,1,POINT (-87.62604 41.87951),1797,G,31180,41.87950,-87.62610,1
161,30383,NaN,Washington/Wabash,None,41.883220,-87.626189,0,41700,1,G,G,1,POINT (-87.62619 41.88322),6779,G,31215,41.88320,-87.62619,1
135,30051,NaN,State/Lake,None,41.885740,-87.627835,0,40260,0,G,G,1,POINT (-87.62784 41.88574),207,G,31240,41.88574,-87.62783,1
139,30075,NaN,Clark/Lake (Elevated),None,41.885737,-87.630886,0,40380,1,G,G,1,POINT (-87.63089 41.88574),357,G,31260,41.88574,-87.63089,1
137,30056,NaN,Ashland/63rd,None,41.778860,-87.663766,0,40290,1,G,G,1,POINT (-87.66377 41.77886),3028,G,41020,41.77895,-87.66377,1


In [60]:
import geopandas as gpd

green_data = green_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = green_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'35-Bronzeville-IIT': [41280, 45440, 45441, 45450, 45460], '43rd': [41240, 45500], '47th-Green': [41220, 45520], '51st': [41200, 45540], 'Adams/Wabash': [31180, 55230, 55260], 'Ashland-Lake': [41498, 41500, 45300], 'Ashland/63rd': [41020, 45790], 'Austin-Green': [41715, 45080], 'California-Green': [41580, 45220], 'Central-Green': [41700, 45100], 'Cermak-McCormick Place': [41315, 45445], 'Cicero-Green': [41660, 45140], 'Clark/Lake (Elevated)': [31260, 55180], 'Clinton-Green': [41460, 45340], 'Conservatory': [41620, 45180], 'East 63rd-Cottage Grove': [41120, 45601], 'Garfield-Green': [41180, 45560], 'Halsted-Green': [41060, 45760], 'Harlem-Green': [41761, 45020], 'Indiana': [41260, 45480], 'Kedzie-Green': [41600, 45200], 'King Drive': [41140, 45600], 'Laramie': [41680, 45120], 'Morgan': [41480, 45320], 'Oak Park-Green': [41760, 45040], 'Pulaski-Green': [41640, 45160], 'Ridgeland': [41740, 45060], 'Roosevelt (Elevated)': [41300, 41301, 51200, 55340], 'State/Lake': [31240, 55195], 'Washin

### ORANGE LINE

In [61]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'orange'
orange_ids = gdf[gdf['line'] == 'O']

# Sort the 'orange' rows by the 'size' column
orange_ids_sorted = orange_ids.sort_values(by='CurrentStationId')

# Print the sorted 'orange' rows
orange_ids_sorted.head(16)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
200,30181,NaN,Midway,None,41.786610,-87.737875,0,40930,1,Org,O,1,POINT (-87.73788 41.78661),221,O,51020,41.78661,-87.73796,1
201,30185,NaN,Pulaski-Orange,None,41.799756,-87.724493,0,40960,1,Org,O,1,POINT (-87.72449 41.79976),596,O,51040,41.79986,-87.72453,1
204,30219,NaN,Kedzie-Orange,None,41.804236,-87.704406,0,41150,1,Org,O,1,POINT (-87.70441 41.80424),1128,O,51060,41.80435,-87.70441,1
197,30060,NaN,Western-Orange,None,41.804546,-87.684019,0,40310,1,Org,O,1,POINT (-87.68402 41.80455),1593,O,51080,41.80468,-87.68402,1
196,30022,NaN,35th/Archer,None,41.829353,-87.680622,0,40120,1,Org,O,1,POINT (-87.68062 41.82935),2397,O,51100,41.82949,-87.68070,1
202,30205,NaN,Ashland-Orange,None,41.839234,-87.665317,0,41060,1,Org,O,1,POINT (-87.66532 41.83923),9274,O,51118,41.83933,-87.66536,1
203,30215,NaN,Halsted-Orange,None,41.846780,-87.648088,0,41130,1,Org,O,1,POINT (-87.64809 41.84678),1049,O,51140,41.84689,-87.64815,1
198,30080,NaN,Roosevelt (Elevated),None,41.867405,-87.626590,0,41400,1,Org,O,1,POINT (-87.62659 41.86740),127044,O,51178,41.86744,-87.62659,1
199,30080,NaN,Roosevelt (Elevated),None,41.867405,-87.626590,0,41400,1,Org,O,1,POINT (-87.62659 41.86740),2112,O,51200,41.86744,-87.62659,1
215,30166,NaN,Harold Washington Library,None,41.876862,-87.628196,0,40850,1,Org,O,5,POINT (-87.62820 41.87686),518,O,55040,41.87691,-87.62820,5


In [62]:
import geopandas as gpd

orange_data = orange_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = orange_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'35th/Archer': [51100, 55440], 'Adams/Wabash': [55230, 55260], 'Ashland-Orange': [51118, 55420], 'Clark/Lake (Elevated)': [55180], 'Halsted-Orange': [51140, 55400], 'Harold Washington Library': [55040], 'Kedzie-Orange': [51060, 55480], 'LaSalle/Van Buren': [55055], 'Midway': [51020, 55518], 'Pulaski-Orange': [51040, 55498], 'Quincy': [55080], 'Roosevelt (Elevated)': [51178, 51200, 55340], 'State/Lake': [55195], 'Washington/Wabash': [55235], 'Washington/Wells': [55095], 'Western-Orange': [51080, 55460]}


### PINK LINE

In [63]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'pink'
pink_ids = gdf[gdf['line'] == 'V']

# Sort the 'pink' rows by the 'size' column
pink_ids_sorted = pink_ids.sort_values(by='CurrentStationId')

# Print the sorted 'pink' rows
pink_ids_sorted.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
591,30200,NaN,Polk,None,41.871551,-87.66953,0,41030,1,Pink,V,5,POINT (-87.66953 41.87155),45616,V,15514,41.87155,-87.66954,5
593,30200,NaN,Polk,None,41.871551,-87.66953,0,41030,1,Pink,V,5,POINT (-87.66953 41.87155),45663,V,15515,41.87155,-87.66954,5
589,30200,NaN,Polk,None,41.871551,-87.66953,0,41030,1,Pink,V,5,POINT (-87.66953 41.87155),45709,V,15516,41.87155,-87.66954,5
590,30200,NaN,Polk,None,41.871551,-87.66953,0,41030,1,Pink,V,5,POINT (-87.66953 41.87155),45755,V,15517,41.87155,-87.66954,5
592,30200,NaN,Polk,None,41.871551,-87.66953,0,41030,1,Pink,V,5,POINT (-87.66953 41.87155),45885,V,15518,41.87155,-87.66954,5


In [64]:
import geopandas as gpd

pink_data = pink_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = pink_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)

{'18th': [61200, 65140], '54th/Cermak': [61020, 65318], 'Adams/Wabash': [55230, 55260], 'Ashland-Lake': [41498, 41500, 45300, 61240, 61242, 61245, 61250, 61255, 61258, 61260], 'California-Cermak': [61140, 65200], 'Central Park': [61100, 65240], 'Cicero-Cermak': [61040, 65300], 'Clark/Lake (Elevated)': [55180], 'Clinton-Green': [41460, 45340], 'Damen-Cermak': [61180, 65160], 'Harold Washington Library': [55040], 'Kedzie-Cermak': [61120, 65220], 'Kostner': [61060, 65280], 'LaSalle/Van Buren': [55055], 'Morgan': [41480, 45320], 'Polk': [15514, 15515, 15516, 15517, 15518, 15519, 61220, 65120], 'Pulaski-Cermak': [61080, 65260], 'Quincy': [55080], 'State/Lake': [55195], 'Washington/Wabash': [55235], 'Washington/Wells': [55095], 'Western-Cermak': [61160, 65180]}


In [65]:
{k: v for k, v in sorted(my_dict.items(), key=lambda item: item[1])}

{'Polk': [15514, 15515, 15516, 15517, 15518, 15519, 61220, 65120],
 'Clinton-Green': [41460, 45340],
 'Morgan': [41480, 45320],
 'Ashland-Lake': [41498,
  41500,
  45300,
  61240,
  61242,
  61245,
  61250,
  61255,
  61258,
  61260],
 'Harold Washington Library': [55040],
 'LaSalle/Van Buren': [55055],
 'Quincy': [55080],
 'Washington/Wells': [55095],
 'Clark/Lake (Elevated)': [55180],
 'State/Lake': [55195],
 'Adams/Wabash': [55230, 55260],
 'Washington/Wabash': [55235],
 '54th/Cermak': [61020, 65318],
 'Cicero-Cermak': [61040, 65300],
 'Kostner': [61060, 65280],
 'Pulaski-Cermak': [61080, 65260],
 'Central Park': [61100, 65240],
 'Kedzie-Cermak': [61120, 65220],
 'California-Cermak': [61140, 65200],
 'Western-Cermak': [61160, 65180],
 'Damen-Cermak': [61180, 65160],
 '18th': [61200, 65140]}

### PURPLE LINE

In [66]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'purple'
purple_ids = gdf[gdf['line'] == 'P']

# Sort the 'purple' rows by the 'size' column
purple_ids_sorted = purple_ids.sort_values(by='CurrentStationId')

# Print the sorted 'purple' rows
purple_ids_sorted.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
226,30090,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,P,P,1,POINT (-87.63392 41.88897),470,P,31291,41.88897,-87.63397,1
225,30090,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,P,P,1,POINT (-87.63392 41.88897),621,P,31340,41.88897,-87.63397,1
242,30137,NaN,Chicago-Brown,None,41.896810,-87.635924,0,40710,1,P,P,1,POINT (-87.63592 41.89681),1383,P,31345,41.89678,-87.63595,1
240,30137,NaN,Chicago-Brown,None,41.896810,-87.635924,0,40710,1,P,P,1,POINT (-87.63592 41.89681),1538,P,31350,41.89678,-87.63595,1
239,30137,NaN,Chicago-Brown,None,41.896810,-87.635924,0,40710,1,P,P,1,POINT (-87.63592 41.89681),4376,P,31358,41.89678,-87.63595,1


In [67]:
pd.set_option("display.max_rows",50)
import geopandas as gpd
import pandas as pd

purple_data = purple_ids_sorted[['stop_name', 'CurrentStationId' ]]

my_dict = purple_ids_sorted.groupby('stop_name')[ 'CurrentStationId'].apply(list).to_dict()
print(my_dict)
purple_data.sort_values("CurrentStationId").head(117)

{'Adams/Wabash': [55230, 55260], 'Armitage': [31382, 31383, 31387, 31390, 31393, 31396, 31398, 31399, 31400, 35360], 'Belmont': [31468, 81500, 85315], 'Central-Purple': [71440, 75038], 'Chicago-Brown': [31345, 31350, 31358, 31360, 35382, 35384, 35386, 35389, 35390, 35394, 35396, 35398, 35399, 35400], 'Clark/Lake (Elevated)': [35430, 55180], 'Davis': [71380, 75100], 'Dempster': [71360, 75120], 'Diversey': [31438, 35320], 'Foster': [71400, 75080], 'Fullerton': [31420, 35340], 'Harold Washington Library': [55040], 'Howard': [71176, 71225, 71235, 71300, 75200, 81642, 81650, 81659, 81662, 81667, 81675], 'LaSalle/Van Buren': [55055], 'Linden': [71455, 75020], 'Main': [71340, 75140], 'Merchandise Mart': [31291, 31340, 35405, 35414, 35418, 35420], 'Noyes': [71420, 75055], 'Quincy': [55080], 'Sedgwick': [31363, 31367, 31369, 31371, 31373, 31377, 31379, 31380, 35363, 35367, 35369, 35374, 35376, 35378, 35379, 35380], 'South Boulevard': [71176, 71225, 71235, 71320, 75160, 81642, 81650, 81659, 8166

,stop_name,CurrentStationId
226,Merchandise Mart,31291
225,Merchandise Mart,31340
242,Chicago-Brown,31345
240,Chicago-Brown,31350
239,Chicago-Brown,31358
...,...,...
333,Wilson-South Outer,85159
337,Wilson-South Outer,85170
331,Wilson-South Outer,85219
330,Wilson-South Outer,85240


In [68]:
{k: v for k, v in sorted(my_dict.items(), key=lambda item: item[1])}

{'Merchandise Mart': [31291, 31340, 35405, 35414, 35418, 35420],
 'Chicago-Brown': [31345,
  31350,
  31358,
  31360,
  35382,
  35384,
  35386,
  35389,
  35390,
  35394,
  35396,
  35398,
  35399,
  35400],
 'Sedgwick': [31363,
  31367,
  31369,
  31371,
  31373,
  31377,
  31379,
  31380,
  35363,
  35367,
  35369,
  35374,
  35376,
  35378,
  35379,
  35380],
 'Armitage': [31382,
  31383,
  31387,
  31390,
  31393,
  31396,
  31398,
  31399,
  31400,
  35360],
 'Fullerton': [31420, 35340],
 'Diversey': [31438, 35320],
 'Wellington': [31455, 35300],
 'Belmont': [31468, 81500, 85315],
 'Clark/Lake (Elevated)': [35430, 55180],
 'Harold Washington Library': [55040],
 'LaSalle/Van Buren': [55055],
 'Quincy': [55080],
 'Washington/Wells': [55095],
 'State/Lake': [55195],
 'Adams/Wabash': [55230, 55260],
 'Washington/Wabash': [55235],
 'Howard': [71176,
  71225,
  71235,
  71300,
  75200,
  81642,
  81650,
  81659,
  81662,
  81667,
  81675],
 'South Boulevard': [71176,
  71225,
  71235,


In [69]:
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 120')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
29,B,2022-03-01 00:00:01,41.97208,-87.76551,120,11602,11700,2.743046,Blu,to,O'Hare,False,ttpm_predslip_blue,
103,B,2022-03-01 00:00:21,41.98232,-87.80890,120,11604,11700,3.151556,Blu,to,O'Hare,False,ttpm_predslip_blue,
176,B,2022-03-01 00:00:41,41.97496,-87.77076,120,11607,11700,2.623162,Blu,to,O'Hare,False,ttpm_predslip_blue,
250,B,2022-03-01 00:01:02,41.97580,-87.77222,120,11608,11700,2.622869,Blu,to,O'Hare,False,ttpm_predslip_blue,
324,B,2022-03-01 00:01:22,41.97800,-87.77609,120,11610,11700,2.623537,Blu,to,O'Hare,False,ttpm_predslip_blue,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91155,B,2022-03-06 01:49:21,41.98384,-87.84917,120,11650,11660,3.178393,Blu,to,Rosemont,False,ttpm_predslip_blue,
91195,B,2022-03-06 01:49:41,41.98351,-87.85939,120,11651,11660,3.176772,Blu,to,Rosemont,False,ttpm_predslip_blue,
91235,B,2022-03-06 01:50:01,41.98366,-87.85574,120,11655,11660,3.181652,Blu,to,Rosemont,False,ttpm_predslip_blue,
91275,B,2022-03-06 01:50:21,41.98363,-87.85659,120,11658,11660,3.181652,Blu,to,Rosemont,False,ttpm_predslip_blue,


In [70]:
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')

edf1 = edf[edf['LineName'] == 'Blu']

edf2 = edf1.sort_values(by='CurrentStationId')

edf2.head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
52790,B,2022-03-05 21:02:02,41.87426,-87.81732,218,11020,11660,6.200427,Blu,to,Rosemont,False,ttpm_predslip_blue,
317714,B,2022-03-02 09:26:01,41.87426,-87.81732,231,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
196164,B,2022-03-01 20:56:41,41.87426,-87.81732,1000,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
317701,B,2022-03-02 09:25:41,41.87426,-87.81732,231,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
376205,B,2022-03-07 18:45:01,41.87426,-87.81732,132,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,


In [71]:
import geopandas as gpd

# Read the GeoPandas file
gdf = gpd.read_parquet('station-id-map.parquet')

# Select the rows with the color 'red'
blue_ids = gdf[gdf['line'] == 'B']

# Sort the 'red' rows by the 'size' column
blue_ids_sorted = blue_ids.sort_values(by='CurrentStationId')

# Print the sorted 'red' rows

blue_ids_sorted.head(60)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
17,30076,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,1,POINT (-87.81732 41.87426),31,B,11020,41.87426,-87.81732,1
52,30189,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,1,POINT (-87.80696 41.87349),476,B,11040,41.87349,-87.80696,1
3,30034,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,1,POINT (-87.79160 41.87211),329,B,11060,41.87211,-87.79160,1
0,30001,NaN,Austin-Blue,None,41.870851,-87.776812,0,40010,0,Blue,B,1,POINT (-87.77681 41.87085),3941,B,11080,41.87089,-87.77681,1
51,30187,NaN,Cicero-Forest Park,None,41.871574,-87.745154,0,40970,0,Blue,B,1,POINT (-87.74515 41.87157),5024,B,11100,41.87161,-87.74515,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,30171,NaN,O'Hare,None,41.977665,-87.904223,0,40890,1,Blue,B,1,POINT (-87.90422 41.97767),2646,B,11662,41.97767,-87.90422,1
41,30171,NaN,O'Hare,None,41.977665,-87.904223,0,40890,1,Blue,B,1,POINT (-87.90422 41.97767),2791,B,11663,41.97767,-87.90422,1
44,30171,NaN,O'Hare,None,41.977665,-87.904223,0,40890,1,Blue,B,1,POINT (-87.90422 41.97767),148334,B,11667,41.97767,-87.90422,1
47,30171,NaN,O'Hare,None,41.977665,-87.904223,0,40890,1,Blue,B,1,POINT (-87.90422 41.97767),47419,B,11673,41.97767,-87.90422,1


In [72]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet').head(5)

In [73]:
station_id_map.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
0,30001,NaN,Austin-Blue,None,41.870851,-87.776812,0,40010,0,Blue,B,1,POINT (-87.77681 41.87085),3941,B,11080,41.87089,-87.77681,1
1,30012,NaN,Belmont-Blue,None,41.938132,-87.712359,0,40060,0,Blue,B,1,POINT (-87.71236 41.93813),1245,B,11520,41.93813,-87.71236,1
2,30014,NaN,Jackson-Blue,None,41.878183,-87.629296,0,40070,1,Blue,B,1,POINT (-87.62930 41.87818),2150,B,11300,41.87818,-87.62930,1
3,30034,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,1,POINT (-87.79160 41.87211),329,B,11060,41.87211,-87.79160,1
4,30042,NaN,Western-Forest Park,None,41.875478,-87.688436,0,40220,0,Blue,B,1,POINT (-87.68844 41.87548),7177,B,11160,41.87553,-87.68844,1
